In [1]:
import os
import sys
import pandas as pd
from importlib import reload

In [2]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer
import pandserv as pds # сервисные функции для пандас

In [3]:
import Connection as cn

In [4]:
spark = cn.spark_app_builder(cn.get_spark_master_ip(),
                             spark_app_name = "pyspark-taxi-forecasting")

INFO:ch:spark_master_ip: 172.18.0.2
DEBUG:ch:starting import SparkSession
INFO:ch:starting building spark app object: pyspark-taxi-forecasting
INFO:ch:builded spark app object: <pyspark.sql.session.SparkSession object at 0x7f9865cb4ed0>


In [5]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 10.3 ms, sys: 30.4 ms, total: 40.7 ms
Wall time: 55.7 s


In [6]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [7]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [43]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';',index_col='Column Name')

table_descr

,Description,Type,Описание,Тип
Column Name,,,,
Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст
Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст
Trip Start Timestamp,When the trip started rounded to the nearest 15 minutes.,Date & Time,Время начала поездки округленное до ближайших 15 минут,Дата и время
Trip End Timestamp,When the trip ended rounded to the nearest 15 minutes.,Date & Time,Время окончания поездки округленное до ближайших 15 минут,Дата и время
Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число
Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число
Pickup Census Tract,The Census Tract where the trip began. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где началась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
Dropoff Census Tract,The Census Tract where the trip ended. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где завершилась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
Pickup Community Area,The Community Area where the trip began. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где началась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число


In [10]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [12]:
# %%time
# describe_result = taxi.describe().toPandas()

CPU times: user 447 ms, sys: 104 ms, total: 551 ms
Wall time: 2min 54s


In [15]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [44]:
describe_result_22 = pd.read_csv('/work/data/2022_describe.csv',sep=',',index_col='summary')

In [46]:
describe_result_22_formatted = describe_result_22.copy()
pds.get_df_formated(describe_result_22_formatted, '`',2, 10)

In [47]:
describe_result_22_formatted.transpose()

summary,count,mean,stddev,min,max
Trip ID,6`382`425,nan,nan,000000bb18,ffffff1aae
Taxi ID,6`382`425,nan,nan,0041f8f0c9,fff84aa08a
Trip Start Timestamp,6`382`425,nan,nan,01/01/2022,12/31/2022
Trip End Timestamp,6`382`213,nan,nan,01/01/2022,12/31/2022
Trip Seconds,6`380`960,1`198.21,1`895.66,0,86`341
Trip Miles,6`382`369,6.19,8.00,0,2`967.54
Pickup Census Tract,2`623`831,17`031`468`160.38,368`945.90,17`031`010`100,17`031`980`100
Dropoff Census Tract,2`675`331,17`031`411`846.86,345`773.49,17`031`010`100,17`031`980`100
Pickup Community Area,5`868`572,32.35,25.20,1,77
Dropoff Community Area,5`748`741,25.84,20.93,1,77


In [59]:
print('{:,}'.format(int(describe_result_22.transpose()['count'].astype(float).max())).replace(',', "'"))

6'382'425
